# <font color="Teal">単語埋め込みの活用</font>

- version 0.1 (2022/2/23)
    - 初版（教材作成：一関高専　小池 敦）

本教材では，単語の分散表現を活用したニューラルネットワークを作成します．  
映画に対するレビューのデータセット [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) を使用して，レビューが肯定的（ポジティブ）な内容か否定的（ネガティブ）な内容かを推定するニューラルネットワークを作ります．  
このデータセットでは，ポジティブなレビューとネガティブなレビューが分けられて格納されているため，これをラベルとして利用します．

このネットワークを用いると，レビューがポジティブかネガティブかを推定できるようになりますが，今回のメインの目的は，このネットワークを用いて分散表現を学習させることです．

この内容は，TensorFlow の[単語埋め込みのチュートリアル](https://www.tensorflow.org/text/guide/word_embeddings)をベースにしています．


# 1. ✏️ <font color="Teal">準備</font>


## 1-1. <font color="Teal">教材の複製</font>

まず，本教材を自分のGoogleドライブに保存します．  
以下の手順を行ってください．


1.   <font color="OrangeRed">Chromeブラウザを利用</font>して，本教材にアクセスする．
2.   「ファイル」→<font color="OrangeRed">「ドライブにコピーを保存」</font>を選択し，自分のGoogleドライブに本教材の複製を作る．


教材中のコードを実行する際は，コードの左側にある実行ボタンを押します．


<font color="RoyalBlue">【実習】$3+5=8$ であることを確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
3 + 5
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
3 + 5

## 1-2. <font color="Teal">GPU利用</font>

本教材では計算時間を短くするためにGPUを利用します．
以下の手順を行ってください．


1.   Colabの「ランタイム」→<font color="OrangeRed">「ランタイムのタイプを変更」</font>で，ハードウェアアクセラレータを「None」から<font color="OrangeRed">「GPU」</font>に変更し保存する．  
（既に「GPU」になっている場合はそのままでよい．）


<font color="RoyalBlue">【実習】GPUが割り当てられていることを確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
!nvidia-smi
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
!nvidia-smi

二重線の下の行に，GPUの名前（Tesra K80, Tesla T4 等）が表示されます．  
"NVIDIA-SMI has failed ・・・" と出力される場合は，上記手順が正しく行えていませんので，再度上記手順を行ってください．

# 2. ✏️ <font color="Teal">映画レビューの感情分析</font>

## 2-1. <font color="Teal">準備</font> 

### 2-1-1. <font color="Teal">ライブラリの読み込み</font> 

<font color="RoyalBlue">【実習】本実習で使用するライブラリをインポートする</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

### 2-1-2. <font color="Teal">データのダウンロード</font> 

<font color="RoyalBlue">【実習】映画レビューのデータセット[Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) をカレントディレクトリにダウンロードして解凍したのち，カレントディレクトリのファイル一覧を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(origin=url,
                                  untar=True, cache_dir=".",
                                  cache_subdir="")
os.listdir(".")
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(origin=url,
                                  untar=True, cache_dir=".",
                                  cache_subdir="")
os.listdir(".")

<font color="RoyalBlue">【実習】`aclImdb`ディレクトリのファイル一覧を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
os.listdir("./aclImdb")
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
os.listdir("./aclImdb")

<font color="RoyalBlue">【実習】`aclImdb`ディレクトリ内にある`README`ファイルを表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
!cat ./aclImdb/README
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
!cat ./aclImdb/README

### 2-1-3. <font color="Teal">不要なディレクトリの削除</font> 

`README`ファイルを見る限り，`train`ディレクトリ内の`unsup`ディレクトリは不要なので，削除します（後述のライブラリにおいて，余計なディレクトリがあると面倒なので，使わないものは削除することにします）

<font color="RoyalBlue">【実習】`./aclImdb/train`ディレクトリのファイル一覧を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
os.listdir("./aclImdb/train")
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
os.listdir("./aclImdb/train")

<font color="RoyalBlue">【実習】`./aclImdb/train`ディレクトリの中から`unsup`ディレクトリを削除する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
shutil.rmtree("./aclImdb/train/unsup")
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
shutil.rmtree("./aclImdb/train/unsup")

<font color="RoyalBlue">【実習】`./aclImdb/train`ディレクトリから`unsup`ディレクトリが削除されたことを確認する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
os.listdir("./aclImdb/train")
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
os.listdir("./aclImdb/train")

## 2-2. <font color="Teal">データセットの生成</font> 

TensorFlowの[Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) クラスを使用すると，データのサイズが大きい場合でも，適切にストレージからメモリへのファイル読み込みを行うことができます．  
ここでは，学習用Datasetと検証用Datasetのふたつを作ります．

### 2-2-1. <font color="Teal">学習データと検証データの作成</font> 

[text_dataset_from_directory](https://www.tensorflow.org/api_docs/python/tf/keras/utils/text_dataset_from_directory) 関数を用いて，ディレクトリから教師ラベル付きのデータを読み込み，データセットを作ります．  
この時，ディレクトリ構成は以下のようにします．

* ラベルごとにディレクトリを作成することで，ラベルごとにデータを格納する
* 1サンプル1ファイルとする

具体的には以下のようになります（`class_a`と`class_b`のふたつのラベルがある場合）．
```
main_directory/
...class_a/
......a_text_1.txt
......a_text_2.txt
...class_b/
......b_text_1.txt
......b_text_2.txt
```

[Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) では学習データセット，テストデータセットとも上記のようなディレクトリ構成になっているため，そのまま読み込みが可能です．

<font color="RoyalBlue">【実習】`train`ディレクトリから，学習用に80%，検証用に20%ずつデータを取得し，データセットを作成する．
* `validation_split`で指定した割合が検証用となる
* `subset`で`"training"`（学習用），`"validation"`（検証用）のどちらを取得するのかを指定する
</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
batch_size = 1024
seed = 123
train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
batch_size = 1024
seed = 123
train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【[Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) では，複数サンプルをひとつのバッチとして管理します．
ここでは1024サンプル（`batch_size`）を1バッチとする設定になっています．
`seed`については，サンプルをシャッフルする際の乱数の種です．】</font>



<font color="RoyalBlue">【実習】最初の5サンプルをラベル`(1: positive, 0: negative)`と共に表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
for text_batch, label_batch in train_ds.take(1):
  for i in range(5):
    print(label_batch[i].numpy(), text_batch.numpy()[i])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(5):
    print(label_batch[i].numpy(), text_batch.numpy()[i])

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【最初のバッチの1024サンプルのうち，先頭5サンプルを表示しています．】</font>



### 2-2-2. <font color="Teal">プリフェッチの設定</font> 

学習中に次に使うサンプルを先読みすることにより，データ読み込みによる待ち時間を減らすことができます．  
このような処理をプリフェッチと呼びます．  
ここでは，プリフェッチに使用するバッファのサイズを環境に応じて自動で調整するようにします．

<font color="RoyalBlue">【実習】プリフェッチのためのバッファサイズを自動チューニングする設定にする</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## 2-3. <font color="Teal">モデルの定義とコンパイル</font> 

### 2-3-1. <font color="Teal">テキストに対する前処理</font> 

[`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization)層を使い，文を 単語ID の列に変換します．その際，英語の小文字化や改行タグの削除も行います．

<font color="RoyalBlue">【実習】文に対し，改行タグの削除や英語の小文字化をし，単語IDの列に変換する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

### 2-3-2. <font color="Teal">モデルの定義</font> 

<font color="RoyalBlue">【実習】モデルを定義する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

<font color="RoyalBlue">【実習】モデルの概要を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model.summary()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
model.summary()

### 2-3-3. <font color="Teal">モデルのコンパイル</font> 

<font color="RoyalBlue">【実習】モデルをコンパイルする</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## 2-4. <font color="Teal">モデルの学習</font> 

### 2-4-1. <font color="Teal">学習</font> 

<font color="RoyalBlue">【実習】モデルを学習させる</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

### 2-4-2. <font color="Teal">学習過程のチェック</font> 

<font color="RoyalBlue">【実習】TensorBoardで学習途中での性能変化を表示する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
%load_ext tensorboard
%tensorboard --logdir logs
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

## 2-5. <font color="Teal">分散表現の抽出</font> 

<font color="RoyalBlue">【実習】[`get_weights`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer#get_weights)メソッドを用いて，単語埋め込み層の重みを抽出する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

<font color="RoyalBlue">【実習】単語の分散表現を`vectors.tsv`に単語リストを`metadata.tsv`に保存する</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

<font color="RoyalBlue">【実習】上記で保存したファイルをダウンロードする</font>

```
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
```

In [ ]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<font color="RoyalBlue">【実習】[Embedding Projector](http://projector.tensorflow.org/)を用いて，分散表現を可視化する
* ツール上で「Load」をクリック
* ファイルとして，`vectors.tsv`と`metadata.tsv`を選択する
</font>

<font color="OrangeRed">(っ´･ω･)っ</font><font color="IndianRed">【今回は学習データサイズが小さいため，解釈性の高い分散表現になっている訳ではありません．】</font>
